Estimação da Pose de um Robô Móvel:

a. determinar a pose (posição e orientação) no plano da imagem do robo movel Husky, utilizar como marcador a caixa vermelha posicionada
sobre o robo.

b. Conceitos como cromaticidade, remoção de ruído, binarizacao,
momentos, elipse equivalente, característica do objeto e circularidade devem ser aplicados para realizar a tarefa.

c. Entregar: o código em Matlab ou outra. (python, c, etc), gráficos com as curvas (u, v e orientação); gráfico da trajetória realizada no plano da imagem;

d. Validar para os dois videos fornecidos

In [ ]:
import cv2 as cv
import numpy as np

# Função para processar cada frame do vídeo
def process_frame(frame, background_remover, path):
    result = frame.copy()
    image = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # Definir o intervalo de cores para detecção do vermelho
    lower_red = np.array([165, 30, 20])
    upper_red = np.array([189, 255, 255])

    # Criar máscara para a área vermelha
    mask = cv.inRange(image, lower_red, upper_red)
    red_area = cv.bitwise_and(result, result, mask=mask)

    # Aplicar desfoque para reduzir ruídos
    frame = cv.GaussianBlur(frame, (5, 5), 0)

    # Remover o fundo e aplicar operações morfológicas para obter contornos
    mask = background_remover.apply(red_area)
    _, mask = cv.threshold(mask, 254, 255, cv.THRESH_BINARY)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Encontrar contornos
    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Encontrar o maior contorno válido e desenhar uma elipse sobre ele
    max_cnt = [0, 0]
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 800 and len(cnt) >= 5:
            if area > max_cnt[0]:
                max_cnt = [area, cnt]

    if max_cnt[0] != 0:
        elipse = cv.fitEllipse(max_cnt[1])
        path.append([int(elipse[0][0]), int(elipse[0][1])])
        cv.ellipse(frame, elipse, (0, 255, 0), 2)

    return frame, path

# Função principal para seguir o robô Husky no vídeo
def follow_husky(input_video, name, outpath):
    cap = cv.VideoCapture(input_video)
    background_remover = cv.createBackgroundSubtractorKNN(detectShadows=True)

    frames = []
    path = []

    # Processamento de cada frame do vídeo
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        # Processar o frame atual
        frame, path = process_frame(frame, background_remover, path)
        frames.append(frame)

    cap.release()

    # Salvar o vídeo com a identificação da caixa vermelha
    save_video(frames, outpath, name)

    # Gerar a imagem da trajetória do marcador
    generate_path_image(path, outpath, name)

# Função para salvar o vídeo identificado com a caixa vermelha
def save_video(frames, outpath, name):
    if frames:
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        out = cv.VideoWriter(outpath + name + '.avi', fourcc, 20.0, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()

# Função para gerar a imagem da trajetória do marcador
def generate_path_image(path, outpath, name):
    if path:
        # Criar uma imagem em branco para desenhar a trajetória
        img = np.zeros((720, 1280, 3), dtype=np.uint8)  # Ajuste o tamanho conforme necessário
        step = 3
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        c_inc = 0

        # Desenhar a trajetória com setas na imagem em branco
        for i in range(10, len(path) - 10, step):
            u = path[i][0] - path[i - step][0]
            v = path[i][1] - path[i - step][1]
            orientation = np.arctan2(v, u)

            # Verificar se o tamanho do vetor é aceitável
            if np.sqrt(u ** 2 + v ** 2) > 50:
                continue

            cv.arrowedLine(img, (path[i - step][0], path[i - step][1]), (path[i][0], path[i][1]),
                           colors[c_inc % 3], 1, tipLength=0.8)
            c_inc += 1

        # Salvar a imagem da trajetória
        cv.imwrite(outpath + name + '.png', img)

# Chamadas para seguir o robô Husky nos vídeos fornecidos


follow_husky('/content/Video1_husky.mp4', 'husky1', '/content/')
follow_husky('/content/video2_husky.mp4', 'husky2', '/content/')
